<a href="https://colab.research.google.com/github/EltonGS/vacinacao_bcg_estados/blob/main/Notebooks/tratamento_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

url_dados = 'https://raw.githubusercontent.com/EltonGS/vacinacao_bcg_estados/main/Dados/A200533189_28_143_208.csv'
url_casos = 'https://raw.githubusercontent.com/EltonGS/vacinacao_bcg_estados/main/Dados/casos_tuberculose.csv'

dados = pd.read_csv(url_dados, encoding='ISO-8859-1', skiprows=4, skipfooter=20, sep=';', thousands='.')
dados_casos = pd.read_csv(url_casos, encoding='ISO-8859-1', skiprows = 3, skipfooter = 19, sep=';', thousands='.')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
# padronizando o nome da coluna do dataset dados para tornar mais fácil o tratamento
dados.rename(columns={'Unidade da Federação' : 'uf'}, inplace=True)

#definindo a coluna com o nome dos estados como o index para o dataset dados
dados.set_index('uf', inplace = True)

# removendo a coluna Total para o dataset dados
dados.drop(columns=['1994', '1995', '1996', 'Total'], inplace = True)

# criando uma nova coluna Total somando todos os valores nas linhas do dataset dados
# dados['Total'] = dados.sum(axis=1)

# removendo os número iniciais a frente da coluna com o nome dos estados para o dataset dados
dados.index = dados.index.str[3:]

# transpondo o dataset casos para os anos serem o index
dados = dados.T

# padronizando o nome da coluna do dataset dados_casos para tornar mais fácil o tratamento
dados_casos.rename(columns={'UF de notificação' : 'uf'}, inplace=True)

3# definindo a coluna com o nome dos estados como o index para o dataset dados_casos
dados_casos.set_index('uf', inplace=True)

# retirando as colunas sem dados do dataset dados_casos
dados_casos.drop(columns=['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1995',
                          '1996', '1997', '1998', '1999', '2021', 'Total'], inplace=True)

# removendo os número iniciais a frente da coluna com o nome dos estados para o dataset dados_casos
dados_casos.index = dados_casos.index.str[3:]

# retirando a linha ignorado/exterior do datase dados_casos
dados_casos.drop(index='Ignorado/exterior', inplace=True)

dados_casos = dados_casos.astype(int)

# criando uma nova coluna Total somando todos os valores nas linhas do dataset dados_casos
# dados_casos['Total'] = dados_casos.sum(axis=1)

# transpondo o dataset dados_casos para que os anos sejam o index
dados_casos = dados_casos.T

In [3]:
# criando uma função para alterar o tipo de dado do index para os 2 datasets, transformando os index em data
def definir_ano (ano: str):
  ano_int = int(ano)
  
  return date(ano_int, 1, 1)

dados.index = dados.index.map(definir_ano)
dados_casos.index = dados_casos.index.map(definir_ano)

In [4]:
dados_casos.head()

uf,Rondônia,Acre,Amazonas,Roraima,Pará,Amapá,Tocantins,Maranhão,Piauí,Ceará,Rio Grande do Norte,Paraíba,Pernambuco,Alagoas,Sergipe,Bahia,Minas Gerais,Espírito Santo,Rio de Janeiro,São Paulo,Paraná,Santa Catarina,Rio Grande do Sul,Mato Grosso do Sul,Mato Grosso,Goiás,Distrito Federal
2001-01-01,662,353,2333,149,3447,216,280,2818,1547,3747,1139,1338,4308,1271,492,7762,1166,1614,16528,18649,3158,1595,5044,957,1359,1111,612
2002-01-01,636,373,2271,172,3764,295,330,2981,1471,4068,1167,1391,4859,1345,590,7726,5875,1596,16779,19867,3320,1851,5487,897,1259,1126,593
2003-01-01,648,370,2333,200,3936,274,275,2987,1400,5228,1356,1451,5168,1401,670,8472,5934,1533,15876,19204,3450,1868,5752,973,1230,1235,613
2004-01-01,623,350,2557,217,4077,277,280,2949,1576,4363,1368,1507,5439,1435,653,8124,6090,1528,15932,18612,3309,1855,5928,1047,1172,1093,626
2005-01-01,628,329,2498,160,3992,269,263,3179,1564,4695,1343,1473,5359,1483,827,7950,5955,1462,15268,17746,3373,1835,5644,1085,1350,1104,619


In [5]:
dados.head()

uf,Rondônia,Acre,Amazonas,Roraima,Pará,Amapá,Tocantins,Maranhão,Piauí,Ceará,Rio Grande do Norte,Paraíba,Pernambuco,Alagoas,Sergipe,Bahia,Minas Gerais,Espírito Santo,Rio de Janeiro,São Paulo,Paraná,Santa Catarina,Rio Grande do Sul,Mato Grosso do Sul,Mato Grosso,Goiás,Distrito Federal
1997-01-01,35505,12384,72388,9237,174889,10189,29557,153225,63535,215827,57042,77338,211107,59381,46970,261086,301189,60761,294463,795671,202587,108326,179670,49702,60070,102276,65566
1998-01-01,37018,13242,73559,8929,128708,12649,31092,155751,64898,166888,52126,72826,204655,65099,47963,270508,280525,63702,284553,793238,207542,90682,174635,47573,57271,101776,60820
1999-01-01,34723,13272,81282,10290,154564,16823,32959,161104,68361,201012,59916,76334,217701,75639,49143,277616,338945,66231,310112,801618,201381,96205,190636,48854,56287,106840,65233
2000-01-01,34654,17101,84148,18261,162481,16755,31203,163096,68393,193388,58329,72677,203858,78991,46739,289827,372241,61956,282577,759875,199414,100736,181810,49227,60344,103027,61784
2001-01-01,35213,20105,84517,10048,183776,26502,32660,172299,68485,185396,57418,80286,194924,73659,44729,311442,327789,60841,258744,701361,187968,91855,167780,48083,57838,102643,57632


In [6]:
# manipulando o dataset dados para que possamos plotar os gráficos mais facilmente
dados = dados.reset_index().melt(id_vars=['index'], value_vars = dados.columns)
dados.columns = ['ano', 'uf', 'n_vacina']
dados['ano'] = dados['ano'].astype('datetime64').dt.year
dados.head()

,ano,uf,n_vacina
0,1997,Rondônia,35505
1,1998,Rondônia,37018
2,1999,Rondônia,34723
3,2000,Rondônia,34654
4,2001,Rondônia,35213


In [7]:
# manipulando o dataset dados_casos para que possamos plotar os gráficos mais facilmente
dados_casos = dados_casos.reset_index().melt(id_vars=['index'], value_vars = dados_casos.columns)
dados_casos.columns = ['ano', 'uf', 'n_casos_tb']
dados_casos['ano'] = dados_casos['ano'].astype('datetime64').dt.year
dados_casos.head()

,ano,uf,n_casos_tb
0,2001,Rondônia,662
1,2002,Rondônia,636
2,2003,Rondônia,648
3,2004,Rondônia,623
4,2005,Rondônia,628


In [12]:
dados.to_csv(r'C:\Users\Elton\Documents\Data Science\projeto_modulo_2\dados_tratados\dados_vac_tb.csv', index=False, encoding='ISO-8859-1')